# Kaggle: Housing Prices

Kaggle housing prices challenge in Julia.  
This is an attempt to complete the kaggle housing prices challenge, described [here](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data), with the Julia language.

I do not have any experience with Julia, so I'm relying on tutorials that can be found on [JuliaBox](https://juliabox.com/) to guide me in data processing, plotting, and analysis.  
This will mirror other work that I have tried for this challenge that uses Python (with which I'm much more familiar).

In [2]:
using Flux    # ML library
using PyPlot  # plotting library with which I'm very familiar, see for example https://github.com/demarley/hepPlotter

The data is stored in this local repository, so let's access it and start looking at it!